### Swap

In [1]:
# Input
date = '2018_09_21'
homePath = 'C:/Users/Mamed/Python4DS/'
projPath = homePath + 'FundsBR/'
strucPath = projPath + 'Structures_' + date + '/'
libsPath = projPath + 'Libs/'

In [2]:
import pandas as pd
import numpy as np
from collections import Counter
import datetime as dt
import matplotlib.pyplot as plt
import matplotlib.cm as cm
from matplotlib import gridspec
from IPython.display import clear_output
import copy

# Load my libraries
exec(open(libsPath + 'fundsLib.py').read())

#### Government bonds (blk 1)

In [3]:
# Read BLC
dfBLC3 = pd.read_pickle(strucPath + 'BLC_blk_3_.pkl')
l0 = dfBLC3.shape[0]
print('{:>12,.0f} rows {:>6,.0f} columns'.format(dfBLC3.shape[0], dfBLC3.shape[1]))

      72,809 rows     18 columns


In [4]:
# Summary
dfSummary(dfBLC3)

RangeIndex: 72809 entries, 0 to 72808
Data columns (total 18 columns):

                    colType  Unique    NaN
TP_FUNDO             object       8      0
CNPJ_FUNDO           object    1826      0
DENOM_SOCIAL         object    1825      0
DT_COMPTC            object     164      0
TP_APLIC             object       2      0
TP_ATIVO             object       1    228
EMISSOR_LIGADO      float64       0  72809
TP_NEGOC             object       2  48692
QT_VENDA_NEGOC      float64       0  72809
VL_VENDA_NEGOC      float64       0  72809
QT_AQUIS_NEGOC      float64       0  72809
VL_AQUIS_NEGOC      float64       0  72809
QT_POS_FINAL        float64       0  72809
VL_MERC_POS_FINAL   float64   70442      0
VL_CUSTO_POS_FINAL  float64   11957  50634
DT_CONFID_APLIC      object     428  28234
CD_SWAP              object      45    228
DS_SWAP              object      45    228


In [5]:
# The confidentiality period (last 3 months) will have null 'CD-SWAPS's,
# i.e., for that dates, we will not be able to see the assets.
# Therefore, we delete those rows.
dfBLC3 = dfBLC3[~dfBLC3['CD_SWAP'].apply(pd.isnull)].reset_index(drop = True)
l1 = dfBLC3.shape[0]
print('Deleted rows: {:<20,.0f}'.format(l0 - l1))

Deleted rows: 228                 


In [7]:
# Two date columns have to be convervet from string to datestamp
dfBLC3['DT_COMPTC'] = pd.to_datetime(dfBLC3['DT_COMPTC'])
dfBLC3['DT_CONFID_APLIC'] = pd.to_datetime(dfBLC3['DT_CONFID_APLIC'])

In [8]:
# Summary
dfSummary(dfBLC3)

RangeIndex: 72581 entries, 0 to 72580
Data columns (total 18 columns):

                           colType  Unique    NaN
TP_FUNDO                    object       8      0
CNPJ_FUNDO                  object    1825      0
DENOM_SOCIAL                object    1824      0
DT_COMPTC           datetime64[ns]     164      0
TP_APLIC                    object       2      0
TP_ATIVO                    object       1      0
EMISSOR_LIGADO             float64       0  72581
TP_NEGOC                    object       2  48464
QT_VENDA_NEGOC             float64       0  72581
VL_VENDA_NEGOC             float64       0  72581
QT_AQUIS_NEGOC             float64       0  72581
VL_AQUIS_NEGOC             float64       0  72581
QT_POS_FINAL               float64       0  72581
VL_MERC_POS_FINAL          float64   70214      0
VL_CUSTO_POS_FINAL         float64   11957  50406
DT_CONFID_APLIC     datetime64[ns]     424  28234
CD_SWAP                     object      45      0
DS_SWAP                     

In [9]:
# Fund type
seriesSummary(dfBLC3, 'TP_FUNDO', 0)

FI          70882
FIF          1402
FMP-FGTS      168
FITVM          77
FIIM           19
FAPI           15
FIP            11
FACFIF          7
Name: TP_FUNDO, dtype: int64

Type:  object
Unique:  8
NaNs:  0


In [12]:
# Investment type
seriesSummary(dfBLC3, 'TP_APLIC', 5)

DIFERENCIAL DE SWAP A RECEBER    38279
DIFERENCIAL DE SWAP A PAGAR      34302
Name: TP_APLIC, dtype: int64

Type:  object
Unique:  2
NaNs:  0


In [20]:
seriesSummary(dfBLC3, 'CD_SWAP', 5)

SDM    18904
SDL    18065
SDP    13400
SDC    11122
SCP     3411
Name: CD_SWAP, dtype: int64

Type:  object
Unique:  45
NaNs:  0


In [21]:
seriesSummary(dfBLC3, 'DS_SWAP', 5)

DI1 X IGM    18904
DI1 X IAP    18065
DI1 X PRE    13400
DI1 X DOL    11122
DOL X PRE     3411
Name: DS_SWAP, dtype: int64

Type:  object
Unique:  45
NaNs:  0


In [26]:
dfRec = dfBLC3[dfBLC3['TP_APLIC'] == 'DIFERENCIAL DE SWAP A RECEBER']
dfPag = dfBLC3[dfBLC3['TP_APLIC'] == 'DIFERENCIAL DE SWAP A PAGAR']
#[['TP_APLIC', 'VL_MERC_POS_FINAL', 'CD_SWAP', 'DS_SWAP']]

In [27]:
seriesSummary(dfRec, 'CD_SWAP', 5)

SDM    9600
SDL    8368
SDP    7112
SDC    6796
SCP    2098
Name: CD_SWAP, dtype: int64

Type:  object
Unique:  43
NaNs:  0


In [28]:
seriesSummary(dfPag, 'CD_SWAP', 5)

SDL    9697
SDM    9304
SDP    6288
SDC    4326
SCP    1313
Name: CD_SWAP, dtype: int64

Type:  object
Unique:  39
NaNs:  0


In [29]:
seriesSummary(dfRec, 'DS_SWAP', 5)

DI1 X IGM    9600
DI1 X IAP    8368
DI1 X PRE    7112
DI1 X DOL    6796
DOL X PRE    2098
Name: DS_SWAP, dtype: int64

Type:  object
Unique:  43
NaNs:  0


In [30]:
seriesSummary(dfPag, 'DS_SWAP', 5)

DI1 X IAP    9697
DI1 X IGM    9304
DI1 X PRE    6288
DI1 X DOL    4326
DOL X PRE    1313
Name: DS_SWAP, dtype: int64

Type:  object
Unique:  39
NaNs:  0


In [42]:
dfRec[dfRec['DT_COMPTC'] == '2017-12-31']['VL_MERC_POS_FINAL'].sum() / 10**9

15.86566962296

In [44]:
dfPag[dfPag['DT_COMPTC'] == '2017-12-31']['VL_MERC_POS_FINAL'].sum() / 10**9

2.866280618480001